# **Carga de los datos**

**Elaborado por:** Jacqueline Fernández, Jazmín Fernández y Nina Odoux.

In [25]:
# Importación de librerías

import pandas as pd
from datetime import datetime
import numpy as np

# **Creación de Tabla de Hechos y de Dimensiones**

In [26]:
# Cargar datos
driver_standings = pd.read_csv('driver_standings_cleaned.csv')
race_details = pd.read_csv('race_details_cleaned.csv')
driver_details = pd.read_csv('driver_details_cleaned.csv')

# Crear IDs únicos para pilotos, coches y carreras
driver_standings['driver_id'] = driver_standings['driver'].factorize()[0]
driver_standings['car_id'] = driver_standings['car'].factorize()[0]

# Mapear driver_id en race_details
driver_standings_unique_driver = driver_standings[['driver', 'driver_id']].drop_duplicates()
race_details['driver_id'] = race_details['driver'].map(driver_standings_unique_driver.set_index('driver')['driver_id'])

# Normalizar nombres de coches en race_details
car_name_mapping = {
    'Era': 'ERA',
    'Alta': 'Alta',
    'Maserati Offenhauser': 'Maserati',
    # Agrega más mapeos aquí según sea necesario
}
race_details['car_normalized'] = race_details['car'].replace(car_name_mapping)

# Generar IDs únicos para coches
driver_standings_unique_car = driver_standings[['car', 'car_id']].drop_duplicates()
unmatched_cars_normalized = race_details.loc[
    race_details['car_normalized'].notna() & ~race_details['car_normalized'].isin(driver_standings['car'])
]['car_normalized'].unique()

# IDs temporales para coches no encontrados
temp_ids = {car: idx + 1000 for idx, car in enumerate(unmatched_cars_normalized)}
race_details['car_id'] = race_details['car_normalized'].map(driver_standings_unique_car.set_index('car')['car_id'])
race_details['car_id'] = race_details['car_id'].fillna(race_details['car_normalized'].map(temp_ids))

# IDs únicos para carreras
race_details['race_id'] = race_details['grand_prix'].factorize()[0]

# Dimensión: Dim_Drivers
dim_drivers = driver_standings[['driver_id', 'driver', 'nationality', 'drivercode']].drop_duplicates(subset=['driver_id'])
number_asigned = race_details[['driver_id', 'no']].drop_duplicates(subset=['driver_id']).set_index('driver_id')['no']
dim_drivers = dim_drivers.merge(number_asigned, on='driver_id', how='left')
dim_drivers.rename(columns={
    'driver': 'driver_name',
    'nationality': 'nationality',
    'drivercode': 'driver_code',
    'no': 'number_asigned'
}, inplace=True)

# Dimensión: Dim_Cars
unique_cars_from_facts = race_details[['car_id', 'car_normalized']].rename(columns={'car_normalized': 'car_name'})
dim_cars = pd.concat([
    driver_standings[['car_id', 'car']].rename(columns={'car': 'car_name'}),
    unique_cars_from_facts
]).drop_duplicates(subset=['car_id'])

# Dimensión: Dim_Races
dim_races = race_details[['race_id', 'grand_prix', 'year', 'detail']].drop_duplicates(subset=['race_id'])
dim_races.rename(columns={
    'grand_prix': 'grand_prix',
    'year': 'year',
    'detail': 'detail'
}, inplace=True)

# Dimensión: Dim_Time
driver_details['date'] = pd.to_datetime(driver_details['date'])
dim_time = driver_details[['date']].drop_duplicates()
dim_time['date_id'] = range(1, len(dim_time) + 1)
dim_time['year'] = dim_time['date'].dt.year
dim_time['month'] = dim_time['date'].dt.month
dim_time['day'] = dim_time['date'].dt.day

# Reordenar las columnas para que date_id sea la primera
dim_time = dim_time[['date_id', 'date', 'year', 'month', 'day']]

# Dimensión: Dim_Historical_Stats
total_points = driver_standings.groupby('driver_id')['pts'].sum().reset_index(name='total_points')
total_races = race_details.groupby('driver_id').size().reset_index(name='total_races')
best_position = race_details.groupby('driver_id')['pos'].min().reset_index(name='best_position')
total_wins = race_details[race_details['pos'] == 1].groupby('driver_id').size().reset_index(name='total_wins')

dim_historical_stats = total_points.merge(total_races, on='driver_id', how='left') \
    .merge(best_position, on='driver_id', how='left') \
    .merge(total_wins, on='driver_id', how='left')
dim_historical_stats['total_wins'] = dim_historical_stats['total_wins'].fillna(0).astype(int)
dim_historical_stats['historical_id'] = range(1, len(dim_historical_stats) + 1)

dim_historical_stats = dim_historical_stats[['historical_id', 'driver_id', 'total_points', 'total_races',
                                             'best_position', 'total_wins']]

# Tabla de Hechos: Fact_Race_Performance
fact_race_performance = race_details[['race_id', 'driver_id', 'car_id', 'year', 'pts', 'pos', 'laps', 'time/retired']]
fact_race_performance.rename(columns={
    'year': 'year',
    'pts': 'points_earned',
    'pos': 'position',
    'laps': 'laps_completed',
    'time/retired': 'time_or_status'
}, inplace=True)

# Validar relaciones en la tabla de hechos
valid_driver_ids = dim_drivers['driver_id'].unique()
fact_race_performance = fact_race_performance[fact_race_performance['driver_id'].isin(valid_driver_ids)]

valid_car_ids = dim_cars['car_id'].unique()
fact_race_performance = fact_race_performance[fact_race_performance['car_id'].isin(valid_car_ids)]

# Guardar tablas resultantes
dim_drivers.to_csv('dim_drivers.csv', index=False)
dim_cars.to_csv('dim_cars.csv', index=False)
dim_races.to_csv('dim_races.csv', index=False)
dim_time.to_csv('dim_time.csv', index=False)
dim_historical_stats.to_csv('dim_historical_stats.csv', index=False)
fact_race_performance.to_csv('fact_race_performance.csv', index=False)

print("Tablas ajustadas, cargadas y guardadas con éxito.")


Tablas ajustadas, cargadas y guardadas con éxito.


<ipython-input-26-2ba3b806fb92>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_race_performance.rename(columns={


# **Validación de carga**

In [27]:
# Cargar la tabla Dim_Time para validación
dim_time = pd.read_csv('dim_time.csv')

# Mostrar las primeras filas para validar la estructura
print("Dim_Time:")
print(dim_time.head())


Dim_Time:
   date_id        date  year  month  day
0        1  1950-05-21  1950      5   21
1        2  1950-06-04  1950      6    4
2        3  1950-06-18  1950      6   18
3        4  1950-09-03  1950      9    3
4        5  1950-05-30  1950      5   30


In [28]:
# Cargar la tabla Dim_Cars para validación
dim_cars = pd.read_csv('dim_cars.csv')

# Mostrar las primeras filas para validar la estructura
print("Dim_Cars:")
print(dim_cars.head())

Dim_Cars:
   car_id                  car_name
0     0.0                Alfa Romeo
1     1.0               Talbot-Lago
2     2.0                   Ferrari
3     3.0  Kurtis Kraft Offenhauser
4     4.0         Deidt Offenhauser


In [29]:
# Cargar la tabla Dim_Drivers para validación
dim_drivers = pd.read_csv('dim_drivers.csv')

# Mostrar las primeras filas para validar la estructura
print("Dim_Drivers:")
print(dim_drivers.head())

Dim_Drivers:
   driver_id         driver_name nationality driver_code  number_asigned
0          0         Nino Farina         ITA         FAR             2.0
1          1  Juan Manuel Fangio         ARG         FAN             1.0
2          2       Luigi Fagioli         ITA         FAG             3.0
3          3        Louis Rosier         FRA         ROS            15.0
4          4      Alberto Ascari         ITA         ASC            40.0


In [30]:
# Cargar la tabla Dim_Historical_Stats para validación
dim_historical_stats = pd.read_csv('dim_historical_stats.csv')

# Mostrar las primeras filas para validar la estructura
print("Dim_Historical_Stats:")
print(dim_historical_stats.head())

Dim_Historical_Stats:
   historical_id  driver_id  total_points  total_races  best_position  \
0              1          0         115.3         33.0           -1.0   
1              2          1         244.5         51.0           -1.0   
2              3          2          28.0          7.0           -1.0   
3              4          3          18.0         38.0           -1.0   
4              5          4         107.6         32.0           -1.0   

   total_wins  
0           5  
1          24  
2           1  
3           0  
4          13  


In [21]:
# Cargar la tabla Dim_Races para validación
dim_races = pd.read_csv('dim_races.csv')

# Mostrar las primeras filas para validar la estructura
print("Dim_Races:")
print(dim_races.head())

Dim_Races:
   race_id        grand_prix  year       detail
0        0     Great Britain  1950  Race-Result
1        1            Monaco  1950  Race-Result
2        2  Indianapolis 500  1950  Race-Result
3        3       Switzerland  1950  Race-Result
4        4           Belgium  1950  Race-Result


In [31]:
# Cargar la tabla de hechos Fact_race_performance para validación
fact_race_performance.csv = pd.read_csv('fact_race_performance.csv')

# Mostrar las primeras filas para validar la estructura
print("Fact_race_performance:")
print(fact_race_performance.head())

Fact_race_performance:
   race_id  driver_id  car_id  year  points_earned  position  laps_completed  \
0        0        0.0     0.0  1950            9.0         1              70   
1        0        2.0     0.0  1950            6.0         2              70   
2        0        8.0     0.0  1950            4.0         3              70   
4        0        3.0     1.0  1950            2.0         5              68   
7        0       17.0     1.0  1950            0.0         8              65   

  time_or_status  
0    2:13:23.600  
1        +2.600s  
2       +52.000s  
4        +2 laps  
7        +5 laps  


<ipython-input-31-ca32d2550ff7>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  fact_race_performance.csv = pd.read_csv('fact_race_performance.csv')


In [32]:
# Verificación 1: Relaciones en la tabla de hechos
# Asegurarse de que no haya IDs de pilotos no referenciados
invalid_driver_references = fact_race_performance[~fact_race_performance['driver_id'].isin(dim_drivers['driver_id'])]
print(f"Invalid Driver References: {len(invalid_driver_references)}")

# Verificación 2: Consistencia de Claves en Dim_Drivers
# Confirma que no haya registros duplicados en la clave primaria
total_drivers = len(dim_drivers)
distinct_drivers = dim_drivers['driver_id'].nunique()
print(f"Total Drivers: {total_drivers}, Distinct Drivers: {distinct_drivers}")
print(f"Duplicated Drivers: {total_drivers - distinct_drivers}")

# Verificación 3: Relaciones entre tabla de hechos y Dim_Cars
# Verificar que todos los IDs de coches en la tabla de hechos existan en Dim_Cars
invalid_car_references = fact_race_performance[~fact_race_performance['car_id'].isin(dim_cars['car_id'])]
print(f"Invalid Car References: {len(invalid_car_references)}")

# Verificación 4: Relaciones entre tabla de hechos y Dim_Races
# Verificar que todos los IDs de carreras en la tabla de hechos existan en Dim_Races
invalid_race_references = fact_race_performance[~fact_race_performance['race_id'].isin(dim_races['race_id'])]
print(f"Invalid Race References: {len(invalid_race_references)}")

# Verificación 5: Consistencia en Dim_Races
# Confirmar que no haya registros duplicados en la clave primaria
total_races = len(dim_races)
distinct_races = dim_races['race_id'].nunique()
print(f"Total Races: {total_races}, Distinct Races: {distinct_races}")
print(f"Duplicated Races: {total_races - distinct_races}")

# Verificación 6: Consistencia de Fechas en Dim_Time
# Confirmar que no haya registros duplicados en la clave primaria
total_dates = len(dim_time)
distinct_dates = dim_time['date_id'].nunique()
print(f"Total Dates: {total_dates}, Distinct Dates: {distinct_dates}")
print(f"Duplicated Dates: {total_dates - distinct_dates}")

# Verificación 7: Relaciones con Dim_Historical_Stats
# Confirmar que todos los IDs de pilotos en Dim_Historical_Stats existan en Dim_Drivers
invalid_historical_stats_references = dim_historical_stats[~dim_historical_stats['driver_id'].isin(dim_drivers['driver_id'])]
print(f"Invalid Historical Stats References: {len(invalid_historical_stats_references)}")

Invalid Driver References: 0
Total Drivers: 394, Distinct Drivers: 394
Duplicated Drivers: 0
Invalid Car References: 0
Invalid Race References: 0
Total Races: 52, Distinct Races: 52
Duplicated Races: 0
Total Dates: 1079, Distinct Dates: 1079
Duplicated Dates: 0
Invalid Historical Stats References: 0
